<a href="https://colab.research.google.com/github/xiekg2020/DL_test/blob/main/3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import tensorflow as tf
from IPython import display
!pip install d2l
from d2l import tensorflow as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [44]:
num_inputs = 784
num_outputs = 10

W = tf.Variable(
    tf.random.normal(shape=(num_inputs, num_outputs), mean=0, stddev=0.01))
b = tf.Variable(tf.zeros(num_outputs))

In [45]:
x=tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
tf.reduce_sum(x,0,keepdims=True),tf.reduce_sum(x,1,keepdims=True)

(<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[5., 7., 9.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[ 6.],
        [15.]], dtype=float32)>)

In [46]:
def softmax(x):
  x_exp=tf.exp(x)
  partition=tf.reduce_sum(x_exp,1,keepdims=True)
  return x_exp/partition

In [47]:
x=tf.random.normal((2,5),0,1)
x_prob=softmax(x)
x_prob,tf.reduce_sum(x_prob,1)

(<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
 array([[0.08884966, 0.02858421, 0.6917096 , 0.12886134, 0.06199522],
        [0.12517199, 0.3782592 , 0.13393122, 0.26261494, 0.10002267]],
       dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.99999994, 1.        ], dtype=float32)>)

In [48]:
def net(x):
  return softmax(tf.matmul(tf.reshape(x,(-1,W.shape[0])),W)+b)

In [49]:
y_hat=tf.constant([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y=tf.constant([0,2])
y_hat.shape[-1],tf.one_hot(y,depth=y_hat.shape[-1]),tf.boolean_mask(y_hat,tf.one_hot(y,depth=y_hat.shape[-1]))

(3, <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 0., 0.],
        [0., 0., 1.]], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.1, 0.5], dtype=float32)>)

In [50]:
def cross_entropy(y_hat,y):
  return -tf.math.log(tf.boolean_mask(y_hat,tf.one_hot(y,depth=y_hat.shape[-1])))

cross_entropy(y_hat,y)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.3025851, 0.6931472], dtype=float32)>

In [51]:
def accuracy(y_hat,y):
  if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
    y_hat=tf.argmax(y_hat,axis=1)
  cmp=tf.cast(y_hat,y.dtype)==y
  return float(tf.reduce_sum(tf.cast(cmp,y.dtype)))

In [52]:
y_hat,y,accuracy(y_hat,y)/len(y)

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[0.1, 0.3, 0.6],
        [0.3, 0.2, 0.5]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 2], dtype=int32)>,
 0.5)

In [71]:
def evaluate_accuracy(net,data_iter):
  metric=Accumulator(2)
  for x,y in data_iter:
    metric.add(accuracy(net(x),y), tf.size(y).numpy())
  return metric[0]/metric[1]

In [54]:
class Accumulator:
  def __init__(self,n):
    self.data=[0.0]*n
  def add(self, *args):
    self.data=[a+float(b) for a,b in zip(self.data, args)]
  def reset(self):
    self.data=[0.0] * len(self.data)
  def __getitem__(self, idx):
    return self.data[idx]

In [73]:
evaluate_accuracy(net,test_iter)

0.0852